<a href="https://colab.research.google.com/github/BrockDSL/ARCH_Data_Explore/blob/main/Municipal_Document_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


![dsl logo](https://github.com/BrockDSL/ARCH_Data_Explore/blob/main/dsl_logo.png?raw=true)


# Municipal Document Similarity

Will load up different snapshots of [URLs of interest](https://github.com/BrockDSL/ARCH_Data_Explore/blob/main/urls_of_interest.txt) and compare similarity using [spaCy tools](https://spacy.io/) for it.


In [1]:

#spaCy tools in separate cell, just to save some download

!pip install spacy==3.2.0
!pip install --upgrade --no-cache-dir gdown
# if doing anything with word vectors use this version
# run this cell then hit ctrl-m-. to restart the runtime then proceed
!python -m spacy download en_core_web_md


     |████████████████████████████████| 6.0 MB 2.4 MB/s 
     |████████████████████████████████| 181 kB 40.0 MB/s 
     |████████████████████████████████| 628 kB 34.7 MB/s 
     |████████████████████████████████| 42 kB 650 kB/s 
     |████████████████████████████████| 10.1 MB 34.7 MB/s 
     |████████████████████████████████| 451 kB 46.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
  Installing build dependencies ... done
  Get

In [ ]:
#restart run-time automatically
import os
os.kill(os.getpid(), 9)

In [1]:
#Install Libraries

import pandas as pd
import spacy
import gdown
from google.colab import files

pd.set_option('display.max_colwidth', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

print("Libraries Loaded")

Libraries Loaded


In [2]:
#Verify package versions

#!pip show gdown
#!pip show spacy

Name: gdown
Version: 4.4.0
Summary: Google Drive direct download of big files.
Home-page: http://github.com/wkentaro/gdown
Author: Kentaro Wada
Author-email: www.kentaro.wada@gmail.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4, six, tqdm, requests, filelock
Required-by: 
Name: spacy
Version: 3.2.0
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: pydantic, langcodes, preshed, spacy-loggers, spacy-legacy, pathy, requests, thinc, tqdm, cymem, packaging, catalogue, numpy, typer, srsly, murmurhash, jinja2, wasabi, typing-extensions, blis, setuptools
Required-by: fastai, en-core-web-sm, en-core-web-md


In [3]:
#SpaCy object that will handle similarity
nlp = spacy.load("en_core_web_md")

In [4]:
#Load dataset for comparison
gdown.download("https://drive.google.com/u/0/uc?id=1oKNphdZkuNfeh-beuTkcIBo_EFLWO9zX&export=download","municipal_data.csv.gz",quiet=False)
!gunzip municipal_data.csv.gz
archive_data = pd.read_csv("municipal_data.csv")
#get rid of some uneeded cols
del(archive_data['Unnamed: 0'])
del(archive_data['index'])
del(archive_data['mime_type_web_server'])
del(archive_data['mime_type_tika'])
del(archive_data['language'])
archive_data['crawl_date']= pd.to_datetime(archive_data['crawl_date'],format='%Y-%m-%d')
print("Data Loaded")

Downloading...
From: https://drive.google.com/u/0/uc?id=1oKNphdZkuNfeh-beuTkcIBo_EFLWO9zX&export=download
To: /content/municipal_data.csv.gz
100%|██████████| 51.3M/51.3M [00:00<00:00, 116MB/s]


Data Loaded


In [42]:
#URLs of interest
url_list = pd.read_csv("https://raw.githubusercontent.com/BrockDSL/ARCH_Data_Explore/main/urls_of_interest.txt",header=None)
url_list.columns = ["base_url"]

In [ ]:
#archive_data.head()


## URL Selection


In [43]:
url_list

,base_url
0,https://www.niagararegion.ca/health/covid-19/default.aspx
1,https://www.niagararegion.ca/health/covid-19/protect-yourself.aspx
2,https://www.niagararegion.ca/health/covid-19/employee-information.aspx
3,https://www.niagararegion.ca/health/covid-19/municipal-bills.aspx
4,https://www.niagararegion.ca/health/covid-19/testing.aspx
5,https://www.niagararegion.ca/health/covid-19/symptoms.aspx
6,https://www.niagararegion.ca/health/covid-19/resources.aspx
7,https://www.niagararegion.ca/health/covid-19/self-isolation.aspx
8,https://www.niagararegion.ca/health/covid-19/social-support.aspx
9,https://www.notl.com/COVID-19/


In [63]:
#@title URLs to Compare
#@markdown enter the number of the first column above to select that cell as your base of comparsion
base_url_choice = 0 #@param {type:"integer"}
#@markdown enter the number of the first column above to select that cell your comparison url
compare_to_choice = 19 #@param {type:"integer"}


In [ ]:
#base_url_time_stamps
archive_data[archive_data['url'] == url_list.iloc[base_url_choice]['base_url']][['crawl_date']]

In [110]:
#@title Base URL Version Choice
#@markdown Choose index of timestamp for which crawl to use

base_url_version_choice = 31069 #@param {type:"integer"}


In [ ]:
archive_data[archive_data['url'] == url_list.iloc[compare_to_choice]['base_url']][['crawl_date']]

In [70]:
#@title COMPARE TO Crawl date

#@markdown Choose index of timestamp for which crawl to use
compare_to_url_version_choice = 28385 #@param {type:"integer"}

In [152]:
#Final creation and comparison

base_url_f = url_list.iloc[base_url_choice]['base_url']
base_url_ts = archive_data.iloc[base_url_version_choice]['crawl_date']

comp_url_f = url_list.iloc[compare_to_choice]['base_url']
comp_url_ts = archive_data.iloc[compare_to_url_version_choice]['crawl_date']

print("Comparison\n")


doc_base = archive_data[archive_data['url']== base_url_f]
doc_base = doc_base[doc_base['crawl_date'] == base_url_ts]
doc_base = doc_base.head(1)
bdate = str(doc_base.crawl_date.values[0]).split('T')[0].split(' ')[0].replace('-','')

print("Base URL: ", base_url_f)
print("Crawl date: ", base_url_ts)
print("IA link: https://web.archive.org/web/" + bdate + "/" + base_url_f)

comp_base = archive_data[archive_data['url']== comp_url_f]
comp_base = comp_base[comp_base['crawl_date'] == comp_url_ts]
comp_base = comp_base.head(1)
cdate = str(comp_base.crawl_date.values[0]).split('T')[0].split(' ')[0].replace('-','')

print("\n")
print("Comp URL: ",comp_url_f)
print("Crawl date: ",comp_url_ts)
print("IA link: https://web.archive.org/web/" + bdate + "/" + comp_url_f)

print("\nSimilarity Score")
db = nlp(doc_base.content.values[0])
dc = nlp(comp_base.content.values[0])

print(db.similarity(dc))

Comparison

Base URL:  https://www.niagararegion.ca/health/covid-19/default.aspx
Crawl date:  2020-11-20 00:00:00
IA link: https://web.archive.org/web/20201120/https://www.niagararegion.ca/health/covid-19/default.aspx


Comp URL:  http://portcolborne.ca/page/covid-19
Crawl date:  2020-11-06 00:00:00
IA link: https://web.archive.org/web/20201120/http://portcolborne.ca/page/covid-19

Similarity Score
0.9780733164429761


In [ ]:
#Run cell to show Base Document
doc_base


In [ ]:
#Run cell to show Compare to Document
comp_base